In [9]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
import time

# Cargar el archivo original
df = pd.read_csv("../Extraction - Detection/Data/text_phishing.csv")
# Verificar nombres de columnas automáticamente
text_col = None
label_col = None

for col in df.columns:
    if "text" in col.lower():
        text_col = col
    if "label" in col.lower() or "type" in col.lower():
        label_col = col

if text_col is None or label_col is None:
    raise ValueError("No se encontraron columnas compatibles con texto y etiqueta.")

# Renombrar columnas
df = df[[text_col, label_col]]
df.columns = ["Email Text", "Email Type"]

# Filtrar solo primeras 6531 filas y correos menores a 5000 caracteres
df = df[df["Email Text"].astype(str).str.len() < 5000].head(6531).copy()
df.reset_index(drop=True, inplace=True)

# Inicializar barra de progreso
tqdm.pandas()

# Función segura de traducción con reintentos
def traducir_seguro(texto, max_reintentos=3):
    for intento in range(max_reintentos):
        try:
            return GoogleTranslator(source='auto', target='es').translate(texto)
        except Exception:
            time.sleep(2)
    return "[TRADUCCIÓN FALLIDA]"

# Traducir con barra de progreso
df["Email Text Spanish"] = df["Email Text"].progress_apply(traducir_seguro)

# Intentar nuevamente las traducciones fallidas
fallidas = df[df["Email Text Spanish"] == "[TRADUCCIÓN FALLIDA]"].index
for i in fallidas:
    df.at[i, "Email Text Spanish"] = traducir_seguro(df.at[i, "Email Text"], max_reintentos=5)

# Guardar como nuevo archivo
df.to_csv("text_phishing_es_new.csv", index=False)
print("Archivo guardado como 'text_phishing_es_new.csv'")

100%|██████████| 6531/6531 [1:39:45<00:00,  1.09it/s]


✅ Archivo guardado como 'text_phishing_es_new.csv'
